In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


# Configuração de estilo para Seaborn (opcional, mas recomendado para estética)
sns.set_theme(style="whitegrid")

# Dicionário de caminhos dos arquivos de dados (assumindo que estão na pasta 'data')
data_paths = {
    'IBGE_PIMPF': '../data/PIMPF - Produção física industrial por seções e atividades industriais.xlsx',
    'CGU_SERVIDORES': '../data/servidorPorOrgao.csv',
    'ATLAS_IDHM': '../data/Atlas.xlsx'  # <-- CORRIGIDO!
}

# Lista de colunas para filtrar o Atlas (para cumprir o limite de atributos do T1)
COLUNAS_ATLAS = [
    'Territorialidades', 'IDHM 2010', 'IDHM Renda 2010', 'IDHM Longevidade 2010', 'IDHM Educação 2010'
]

print("--- INICIANDO ANÁLISE EXPLORATÓRIA DE DADOS (EDA) ---")

# --- 1. FUNÇÕES DE CARREGAMENTO E PREPARAÇÃO ---

# --- 1. FUNÇÕES DE CARREGAMENTO E PREPARAÇÃO ---

def carregar_e_preparar_dados():
    """Carrega, limpa e retorna os DataFrames para visualização."""
    dataframes = {}
    
    # 1.1 CARREGAMENTO IBGE (PIMPF)
    try:
        # 1. Carrega com o header correto (linha 4 = índice 3)
        df_ibge = pd.read_excel(data_paths['IBGE_PIMPF'], header=3) 
        
        # 2. LIMPEZA DOS NOMES DE COLUNAS: Garante que não há espaços extras nas extremidades
        df_ibge.columns = df_ibge.columns.str.strip()
        
        # 3. Renomeia as colunas usando os nomes limpos
        df_ibge = df_ibge.rename(columns={
            'Atividades': 'Seções e atividades industriais',
            # Usaremos 'Ajuste sazonal' ou o nome mais limpo
            'Ajuste sazonal': 'Variação percentual mês/mês imediatamente anterior (com ajuste sazonal)'
        })
        
        # 4. Continua o processo de limpeza e filtragem
        df_ibge = df_ibge[df_ibge['Seções e atividades industriais'].astype(str).str.contains(r'^\d+\. ')]
        
        # Foca nas duas colunas principais para o gráfico 
        df_ibge = df_ibge[['Seções e atividades industriais', 'Variação percentual mês/mês imediatamente anterior (com ajuste sazonal)']]
        df_ibge.dropna(inplace=True)
        dataframes['IBGE_PIMPF'] = df_ibge
        print("✅ Dados IBGE carregados e preparados.")
    except Exception as e:
        print(f"❌ Erro ao carregar IBGE: {e}")

    # 1.2 CARREGAMENTO CGU (SERVIDORES) - (MANTIDO INALTERADO)
    try:
        df_cgu = pd.read_csv(data_paths['CGU_SERVIDORES'], delimiter=';')
        # Remove coluna de lixo e linhas com 'Sem informação'
        df_cgu = df_cgu.drop(columns=['Unnamed: 5'])
        df_cgu.rename(columns={'Servidores': 'Ativos'}, inplace=True)
        df_cgu = df_cgu[df_cgu['Órgão Superior Lotação'] != 'Sem informação']
        
        # Agrupa por Órgão Superior e soma a contagem de servidores
        df_cgu_agrupado = df_cgu.groupby('Órgão Superior Lotação')[['Ativos', 'Inativos', 'Pensionistas']].sum().reset_index()
        # Calcula o total de servidores para gráfico de pizza
        df_cgu_agrupado['Total'] = df_cgu_agrupado[['Ativos', 'Inativos', 'Pensionistas']].sum(axis=1)
        
        # Filtra para mostrar apenas os 5 maiores (melhor para visualização)
        df_cgu_top5 = df_cgu_agrupado.nlargest(5, 'Total')
        dataframes['CGU_SERVIDORES'] = df_cgu_top5
        print("✅ Dados CGU carregados e preparados.")
    except Exception as e:
        print(f"❌ Erro ao carregar CGU: {e}")

    # 1.3 CARREGAMENTO ATLAS (IDHM)
    try:
        # CORREÇÃO: Adiciona engine='python' e especifica o quotechar='"' para lidar com linhas inconsistentes.
        # Mantém encoding='latin-1' para o problema de codificação.
        df_atlas = pd.read_csv(
            data_paths['ATLAS_IDHM'], 
            delimiter=';', 
            encoding='latin-1', 
            engine='python',    # Mais robusto para parsing de arquivos inconsistentes
            quotechar='"'       # Assume que aspas duplas são usadas para agrupar campos com o delimitador interno
        )
        df_atlas = df_atlas[COLUNAS_ATLAS]
        # Renomeia colunas para facilitar a plotagem
        df_atlas.columns = ['Territorialidade', 'IDHM_2010', 'Renda_2010', 'Longevidade_2010', 'Educacao_2010']
        df_atlas.dropna(inplace=True)
        # Exclui a linha do Brasil para focar apenas nos estados/regiões
        df_atlas = df_atlas[df_atlas['Territorialidade'] != 'Brasil']
        dataframes['ATLAS_IDHM'] = df_atlas
        print("✅ Dados ATLAS carregados e preparados.")
    except Exception as e:
        print(f"❌ Erro ao carregar ATLAS: {e}")
        
    return dataframes
    
dataframes = carregar_e_preparar_dados()


# --- 2. GERAÇÃO DOS GRÁFICOS ---

print("\n--- GERANDO VISUALIZAÇÕES ---")

# 2.1 GRÁFICO 1: IBGE - GRÁFICO DE BARRAS (Variação Industrial)
if 'IBGE_PIMPF' in dataframes:
    plt.figure(figsize=(12, 6))
    df_ibge = dataframes['IBGE_PIMPF']
    
    # Ordena para melhor visualização
    df_ibge = df_ibge.sort_values(by='Variação percentual mês/mês imediatamente anterior (com ajuste sazonal)', ascending=False)
    
    # Mapeia cores para destacar valores positivos e negativos
    cores = ['g' if x >= 0 else 'r' for x in df_ibge['Variação percentual mês/mês imediatamente anterior (com ajuste sazonal)']]
    
    sns.barplot(
        x='Variação percentual mês/mês imediatamente anterior (com ajuste sazonal)', 
        y='Seções e atividades industriais', 
        data=df_ibge, 
        palette=cores, 
        orient='h'
    )
    plt.title('Variação Industrial Mensal por Setor (IBGE PIMPF)', fontsize=16)
    plt.xlabel('Variação Percentual Mês/Mês (%)', fontsize=12)
    plt.ylabel('Setor Industrial', fontsize=12)
    plt.tight_layout()
    plt.show()
    print("Gráfico 1: Barras - Variação Industrial (IBGE) gerado.")


# 2.2 GRÁFICO 2: CGU - GRÁFICO DE PIZZA (Distribuição Servidores - Top 5)
if 'CGU_SERVIDORES' in dataframes:
    plt.figure(figsize=(8, 8))
    df_cgu_top5 = dataframes['CGU_SERVIDORES']
    
    # Foca apenas nos 5 maiores órgãos para a pizza
    plt.pie(
        df_cgu_top5['Total'], 
        labels=df_cgu_top5['Órgão Superior Lotação'], 
        autopct='%1.1f%%', 
        startangle=90,
        textprops={'fontsize': 10}
    )
    plt.title('Distribuição de Servidores (Ativos/Inativos/Pensionistas) nos 5 Maiores Órgãos', fontsize=14)
    plt.tight_layout()
    plt.show()
    print("Gráfico 2: Pizza - Distribuição de Servidores (CGU) gerado.")


# 2.3 GRÁFICO 3: ATLAS - GRÁFICO DE DISPERSÃO (Renda vs Longevidade)
if 'ATLAS_IDHM' in dataframes:
    plt.figure(figsize=(10, 7))
    df_atlas = dataframes['ATLAS_IDHM']
    
    # Gráfico de dispersão usando Seaborn para mostrar a relação entre duas dimensões do IDHM
    sns.scatterplot(
        x='Renda_2010', 
        y='Longevidade_2010', 
        data=df_atlas, 
        hue='Territorialidade', # Usa a Territorialidade para diferenciar os pontos
        size='IDHM_2010',       # Usa o IDHM geral para definir o tamanho da bolha
        sizes=(50, 500),        # Define o range de tamanho das bolhas
        alpha=0.7               # Transparência
    )
    
    plt.title('Relação entre IDHM Renda e IDHM Longevidade por Estado (2010)', fontsize=16)
    plt.xlabel('IDHM Renda (2010)', fontsize=12)
    plt.ylabel('IDHM Longevidade (2010)', fontsize=12)
    plt.legend(title='Estado/Região', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout(rect=(0, 0, 0.85, 1)) # Ajusta o layout para a legenda
    plt.show()
    print("Gráfico 3: Dispersão - Renda vs Longevidade (Atlas) gerado.")

print("\n--- EDA CONCLUÍDA ---")